In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import LSTM

from datetime import datetime, timedelta

In [22]:
from neuralforecast.losses.pytorch import MQLoss

In [4]:
AutoLSTM?

Init signature:
AutoLSTM(
    h,
    loss=MAE(),
    valid_loss=None,
    config=None,
    search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator object at 0x7fc9132ba850>,
    num_samples=10,
    refit_with_val=False,
    cpus=40,
    gpus=1,
    verbose=False,
    alias=None,
    backend='ray',
    callbacks=None,
)
Docstring:     
Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

Parameters
----------
cls_model : PyTorch/PyTorchLightning model
    See `neuralforecast.models` [collectio

In [35]:
class FixedModelLSTMProcessor:
    def __init__(self, overall_df, dates):
        self.overall_df = overall_df
        self.overall_df_value_col = "value"
        self.dates = dates
        self.dfs = []
        self.forecasts = []
        self.plotting_df = pd.DataFrame()
        
        self.maes = []
        self.mses = []
        self.mapes = []
        self.nmses = []
        
        self.metrics_df = pd.DataFrame(columns = ["Reference Date", "MAE", "MSE", "MAPE", "NMSE"])
        self.display_df = pd.DataFrame(columns = ["Reference Date", "Target End Date", "Prediction"])
    
    def create_training_dfs(self, value_col):
        self.overall_df_value_col = value_col
        for date in self.dates:
            df = self.overall_df.loc[:date]
            df['ds'] = df.index
            df["unique_id"] = "series_1"
            df = df.rename(columns = {value_col: "y"})
            self.dfs.append(df)
        
    
    def create_fixed_model(self, h, model_name, level = []):
        #Creating AutoLSTM model and predicting with hyperparameter tuning by optuna backend. This is based upon the first training dataframe
        initial_dataset, *_ = TimeSeriesDataset.from_df(self.dfs[0])
        if not level:
            LSTMmodel = AutoLSTM(h = h, backend = "optuna")
            LSTMmodel.fit(dataset = initial_dataset)
        else:
            LSTMmodel = AutoLSTM(h = h, backend = "optuna", loss = MQLoss(level = level))
            LSTMmodel.fit(dataset = initial_dataset)
        
        for i in range(len(self.dfs)):
            dataset, *_ = TimeSeriesDataset.from_df(self.dfs[i])
            y_hat = LSTMmodel.predict(dataset = dataset)
            start_date = datetime.strptime(self.dates[i], "%Y-%m-%d")
            horizon_dates = [start_date + timedelta(weeks=j) for j in range(1, h+1)]
            
            
            y_hat = np.array(y_hat)
            if y_hat.ndim == 1:
                y_hat = y_hat.reshape(-1, 1)

            if levels:
                column_names = ["LSTM-Median", f"LSTM-lo-{levels[-1]}", f"LSTM-lo-{levels[0]}", f"LSTM-hi-{levels[0]}", f"LSTM-hi-{levels[-1]}"]
            else:
                column_names = ["LSTM-Median"]
            fc = pd.DataFrame(y_hat, index=pd.to_datetime(horizon_dates), columns=column_names)

            self.forecasts.append(fc)
        
        LSTMmodel.save(path=f'./AutoLSTM/fixed_models/{model_name}/')
    
    def create_graph(self):
        self.plotting_df.index = self.overall_df.index
        self.plotting_df["Real Data"] = self.overall_df[self.overall_df_value_col]
        
        for i in range(len(self.forecasts)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            
            self.plotting_df[f"{english_date} Model"] = self.forecasts[i]["LSTM-Median"]
        
        self.plotting_df.iplot(xTitle = "Date", yTitle = "Count", title = "Fixed Parameter LSTM Predictions")
    
    def calculate_metrics(self):
        for i in range(len(self.forecasts)):
            mae = mean_absolute_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            mse = mean_squared_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            mape = mean_absolute_percentage_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            nmse = mse/np.var(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index])
            
            self.maes.append(mae)
            self.mses.append(mse)
            self.mapes.append(mape)
            self.nmses.append(nmse)
    
    def display_maes(self):
        for i in range(len(self.maes)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Absolute Error for {english_date} model: {self.maes[i]}")
        
    def display_mses(self):
        for i in range(len(self.mses)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Squared Error for {english_date} model: {self.mses[i]}")
    
    def display_mapes(self):
        for i in range(len(self.mapes)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Absolute Percentage Error for {english_date} model: {self.mapes[i]}")
    
    def display_nmses(self):
        for i in range(len(self.nmses)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Normalized Mean Square Error for {english_date} model: {self.nmses[i]}")
    
    def create_metrics_df(self):
        for i in range(len(self.dates)):
            self.metrics_df.loc[len(self.metrics_df)] = [self.dates[i], self.maes[i], self.mses[i], self.mapes[i], self.nmses[i]]
        
            
    def create_display_df(self):
        for i in range(len(self.forecasts)):
            for index, row in self.forecasts[i].iterrows():
                reference_date = self.dates[i]
                target_end_date = index
                value = row[0]
                self.display_df.loc[len(self.display_df)] = [reference_date, target_end_date, value]
                
            

In [36]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")

In [37]:
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)

In [38]:
Processor = FixedModelLSTMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])

In [39]:
Processor.create_training_dfs(value_col = "value")

In [40]:
Processor.create_fixed_model(h = 4, model_name = "test_model", levels = [80, 95])

[I 2025-04-01 17:08:41,900] A new study created in memory with name: no-name-d9d426ef-68fb-4ae1-bec5-48e3c745500c
/home/hmf6av/.local/lib/python3.11/site-packages/neuralforecast/common/_base_auto.py:291: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[rank: 0] Seed set to 7
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MQLoss        | 5      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | LSTM          | 1.1 M  | train
4 | context_adapter | Linear        | 60.2 K | train
5 | mlp_decoder     | MLP    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [41]:
Processor.forecasts[2]

,LSTM-Median,LSTM-lo-95,LSTM-lo-80,LSTM-hi-80,LSTM-hi-95
2024-11-09,565.854248,376.104614,488.344299,684.059326,738.332581
2024-11-16,524.021484,420.966553,443.558777,660.987000,720.904724
2024-11-23,529.932190,409.866760,448.480347,634.575745,725.592896
2024-11-30,495.173828,430.132263,411.099365,601.863647,655.011841


In [ ]:
Processor.create_graph()

In [ ]:
Processor.plotting_df.iloc[-16:]

In [ ]:
Processor.forecasts[1]

In [ ]:
Processor.overall_df

In [ ]:
Processor.overall_df["value"].loc[Processor.forecasts[1].index]

In [ ]:
Processor.calculate_metrics()

In [ ]:
Processor.maes

In [ ]:
Processor.mses

In [ ]:
Processor.mapes

In [ ]:
Processor.create_display_df()

In [ ]:
Processor.display_df

In [ ]:
Processor.create_metrics_df()

In [ ]:
Processor.metrics_df

In [ ]:
class UpdatingModelLSTMProcessor:
    def __init__(self, overall_df, dates):
        self.overall_df = overall_df
        self.overall_df_value_col = "value"
        self.dates = dates
        self.dfs = []
        self.forecasts = []
        self.plotting_df = pd.DataFrame()
        
        self.maes = []
        self.mses = []
        self.mapes = []
        self.nmses = []
        
        self.metrics_df = pd.DataFrame(columns = ["Reference Date", "MAE", "MSE", "MAPE", "NMSE"])
        self.display_df = pd.DataFrame(columns = ["Reference Date", "Target End Date", "Prediction"])
    
    def create_training_dfs(self, value_col):
        self.overall_df_value_col = value_col
        for date in self.dates:
            df = self.overall_df.loc[:date]
            df['ds'] = df.index
            df["unique_id"] = "series_1"
            df = df.rename(columns = {value_col: "y"})
            self.dfs.append(df)
    
    def create_models(self, h, model_names, levels = []):
        for i in range(len(self.dfs)):
            dataset, *_ = TimeSeriesDataset.from_df(self.dfs[i])
            if not levels:
                LSTMmodel = AutoLSTM(h = h, backend = "optuna")
                LSTMmodel.fit(dataset = dataset)
            else:
                LSTMmodel = AutoLSTM(h = h, backend = "optuna", loss = MQLoss(level = level))
                LSTMmodel.fit(dataset = initial_dataset)
            
            y_hat = LSTMmodel.predict(dataset = dataset)
            start_date = datetime.strptime(self.dates[i], "%Y-%m-%d")
            horizon_dates = [start_date + timedelta(weeks=j) for j in range(1, h+1)]
            
            y_hat = np.array(y_hat)
            if y_hat.ndim == 1:
                y_hat = y_hat.reshape(-1, 1)
                
                
            if levels:
                column_names = ["LSTM-Median", f"LSTM-lo-{levels[-1]}", f"LSTM-lo-{levels[0]}", f"LSTM-hi-{levels[0]}", f"LSTM-hi-{levels[-1]}"]
            else:
                column_names = ["LSTM-Median"]
            fc = pd.DataFrame(y_hat, index=pd.to_datetime(horizon_dates), columns=column_names)

            self.forecasts.append(fc)
            LSTMmodel.save(path=f'./AutoLSTM/updating_models/{model_names[i]}.ckpt')
    
    def create_graph(self):
        self.plotting_df.index = self.overall_df.index
        self.plotting_df["Real Data"] = self.overall_df[self.overall_df_value_col]
        
        for i in range(len(self.forecasts)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            
            self.plotting_df[f"{english_date} Model"] = self.forecasts[i]["LSTM-Median"]
        
        self.plotting_df.iplot(xTitle = "Date", yTitle = "Count", title = "Updating Parameter LSTM Predictions")
    
    def calculate_metrics(self):
        for i in range(len(self.forecasts)):
            mae = mean_absolute_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            mse = mean_squared_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            mape = mean_absolute_percentage_error(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index], self.forecasts[i][0])
            nmse = mse/np.var(self.overall_df[self.overall_df_value_col].loc[self.forecasts[i].index])
            
            self.maes.append(mae)
            self.mses.append(mse)
            self.mapes.append(mape)
            self.nmses.append(nmse)
    
    def display_maes(self):
        for i in range(len(self.maes)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Absolute Error for {english_date} model: {self.maes[i]}")
        
    def display_mses(self):
        for i in range(len(self.mses)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Squared Error for {english_date} model: {self.mses[i]}")
    
    def display_mapes(self):
        for i in range(len(self.mapes)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Mean Absolute Percentage Error for {english_date} model: {self.mapes[i]}")
    
    def display_nmses(self):
        for i in range(len(self.nmses)):
            date_obj = datetime.strptime(self.dates[i], "%Y-%m-%d")
            english_date = date_obj.strftime("%B %d, %Y")
            print(f"Normalized Mean Square Error for {english_date} model: {self.nmses[i]}")
        
    def create_metrics_df(self):
        for i in range(len(self.dates)):
            self.metrics_df.loc[len(self.metrics_df)] = [self.dates[i], self.maes[i], self.mses[i], self.mapes[i], self.nmses[i]]
    
    def create_display_df(self):
        for i in range(len(self.forecasts)):
            for index, row in self.forecasts[i].iterrows():
                reference_date = self.dates[i]
                target_end_date = index
                value = row[0]
                self.display_df.loc[len(self.display_df)] = [reference_date, target_end_date, value]
            
            

In [ ]:
UpdatingProcessor = UpdatingModelLSTMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])

In [ ]:
UpdatingProcessor.dates

In [ ]:
UpdatingProcessor.dfs

In [ ]:
UpdatingProcessor.create_training_dfs(value_col = "value")

In [ ]:
UpdatingProcessor.dfs[2]

In [ ]:
UpdatingProcessor.create_models(h = 4, model_names = ["test_1", "test_2", "test_3", "test_4", "test_5"])

In [ ]:
UpdatingProcessor.forecasts[1]

In [ ]:
UpdatingProcessor.create_graph()

In [ ]:
UpdatingProcessor.calculate_metrics()

In [ ]:
UpdatingProcessor.maes

In [ ]:
UpdatingProcessor.create_display_df()

In [ ]:
UpdatingProcessor.display_df

In [ ]:
UpdatingProcessor.create_metrics_df()

In [ ]:
UpdatingProcessor.metrics_df

In [ ]:
Processor.metrics_df

In [6]:
LSTMmodel = AutoLSTM(h = 4, backend = "optuna")

In [7]:
LSTMmodel.load(path=f'./AutoLSTM/updating_models/test_model.ckpt')

AttributeError: 'AutoLSTM' object has no attribute 'load'

In [43]:
nf = NeuralForecast(models = [LSTM(h = 4)], freq = "W-SAT")

[rank: 0] Seed set to 1


In [46]:
nf.load(path=f'./AutoLSTM/fixed_models/test_model')

NotADirectoryError: [Errno 20] Not a directory: '/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons/AutoLSTM/fixed_models/test_model'

In [45]:
AutoLSTM.save?

Signature: AutoLSTM.save(self, path)
Docstring:
BaseAuto.save

Save the fitted model to disk.

**Parameters:**<br>
`path`: str, path to save the model.<br>
File:      ~/.local/lib/python3.11/site-packages/neuralforecast/common/_base_auto.py
Type:      function

In [47]:
!pwd

/sfs/gpfs/tardis/home/hmf6av/TimeSeriesComparisons


In [48]:
!ls

 ARIMAPipelines.ipynb	        lightning_logs
 AutoARIMA		        LSTMHyperParameter.ipynb
 AutoLSTM		        LSTMPipelines.ipynb
 AutoLSTM.ipynb		        TimeGPT-ARIMA-LSTM-TFT.ipynb
 CESPipelines.ipynb	       'TimeGPT Notebook.ipynb'
 checkpoints		        Untitled.ipynb
 ETSPipelines.ipynb	       'Updated Window Experiments.ipynb'
 HyperParamOptimization.ipynb   WindowExperiments.ipynb
